In [49]:
import csv
import glob
import json
import librosa
import os

import msaf
from msaf import jams2
from msaf.input_output import FileStruct

In [50]:
# Set dataset paths
ds_dir = ".."
original_ref_dir = "../original_references/"

# Get audio files
audio_files = glob.glob(os.path.join(ds_dir, msaf.Dataset.audio_dir, "*.mp3"))

# Make sure that the references folder exist
msaf.utils.ensure_dir(os.path.join(ds_dir, msaf.Dataset.references_dir))

In [53]:
def get_duration(audio_file):
    y, sr = librosa.load(audio_file)
    return len(y) / float(sr)

def fill_global_metadata(jam, csv_file, dur):
    """Fills the global metada into the JAMS jam."""
    jam.metadata.artist = "Sargon"
    jam.metadata.duration = dur  # In seconds
    jam.metadata.title = os.path.basename(csv_file).split("-")[2].split(".")[0]
    jam.metadata.genre = "metal"


def fill_annotation_metadata(annot, attribute):
    """Fills the annotation metadata."""
    annot.annotation_metadata.attribute = attribute
    annot.annotation_metadata.corpus = "Sargon"
    annot.annotation_metadata.version = "1.0"
    annot.annotation_metadata.annotation_tools = "Sonic Visualizer"
    annot.annotation_metadata.annotation_rules = "SALAMI guidelines"
    annot.annotation_metadata.validation_and_reliability = "TODO"
    annot.annotation_metadata.origin = "Jarlem All Studios"
    annot.annotation_metadata.annotator.name = "Oriol Nieto"
    annot.annotation_metadata.annotator.email = "oriol.nieto@gmail.com"


def fill_section_annotation(csv_file, annot):
    """Fills the JAMS annot annotation given a csv file."""

    # Annotation Metadata
    fill_annotation_metadata(annot, "sections")

    with open(orig_ref) as csv_file:
        bounds = list(csv.reader(csv_file, delimiter=','))
        
        for i, bound in enumerate(bounds[:-1]):
            section = annot.create_datapoint()
            section.start.value = float(bound[0])
            section.start.confidence = 0.0
            section.end.value = float(bounds[i+1][0])
            section.end.confidence = 0.0
            section.label.value = str(bound[1])
            section.label.confidence = 1.0
            section.label.context = "function"  # Only function level of annotation

In [ ]:
# Parse CSV
for audio_file in audio_files:
    file_struct = FileStruct(audio_file)
    orig_ref = os.path.join(original_ref_dir, os.path.basename(audio_file)[:-3] + "csv")
    
    # New JAMS and annotation
    jam = jams2.Jams()

    # Global file metadata
    fill_global_metadata(jam, orig_ref, get_duration(audio_file))
    
    # Create Section annotations
    annot = jam.sections.create_annotation()
    fill_section_annotation(orig_ref, annot)
        
    # Save JAMS
    with open(file_struct.ref_file, "w") as jamsfile:
        json.dump(jam, jamsfile, indent=2)